# Einführung in die Verarbeitung von Excel-Dateien mit openpyxl

In diesem Notebook lernst du, wie man Excel-Dateien mit **openpyxl** liest, erstellt und bearbeitet. Außerdem zeigen wir, wie man mehrere Excel-Dateien automatisiert mit **glob** oder **os** verarbeiten kann.

Voraussetzung: Grundkenntnisse in Pandas.

## 1. Installation der benötigten Bibliotheken

Wir benötigen:
- `openpyxl` (für Excel-Dateien im xlsx-Format)
- `pandas` (Datenanalyse)
- `glob` oder `os` (Dateiverarbeitung)

In [1]:
%pip install openpyxl pandas

Note: you may need to restart the kernel to use updated packages.


## 2. Eine Excel-Datei mit openpyxl erstellen

In [2]:
from openpyxl import Workbook

# Neues Excel Workbook erstellen
wb = Workbook()

# Aktives Arbeitsblatt auswählen
ws = wb.active
ws.title = "Verkaufsdaten"

# Einige Daten einfügen
ws.append(['Datum', 'Produkt', 'Kategorie', 'Preis', 'Menge'])
ws.append(['2024-01-01', 'Laptop', 'Elektronik', 1200, 1])
ws.append(['2024-01-02', 'Maus', 'Zubehör', 25, 10])

# Datei speichern
wb.save('verkaufsdaten.xlsx')
print("Excel-Datei 'verkaufsdaten.xlsx' wurde erstellt.")

Excel-Datei 'verkaufsdaten.xlsx' wurde erstellt.


## 3. Eine Excel-Datei mit openpyxl lesen

In [3]:
from openpyxl import load_workbook

# Workbook laden
wb = load_workbook('verkaufsdaten.xlsx')
ws = wb['Verkaufsdaten']

# Alle Zeilen auslesen
for row in ws.iter_rows(values_only=True):
    print(row)

('Datum', 'Produkt', 'Kategorie', 'Preis', 'Menge')
('2024-01-01', 'Laptop', 'Elektronik', 1200, 1)
('2024-01-02', 'Maus', 'Zubehör', 25, 10)


## 4. Excel-Daten in Pandas DataFrame einlesen

Openpyxl kann mit Pandas kombiniert werden, um Excel-Daten direkt in ein DataFrame zu laden.

In [4]:
import pandas as pd

df = pd.read_excel('verkaufsdaten.xlsx', engine='openpyxl')
print(df.head())

        Datum Produkt   Kategorie  Preis  Menge
0  2024-01-01  Laptop  Elektronik   1200      1
1  2024-01-02    Maus     Zubehör     25     10


## 5. Mehrere Excel-Dateien automatisch verarbeiten

Oft hat man viele Excel-Dateien in einem Ordner und möchte sie zusammenführen oder einzeln analysieren.
Dafür nutzen wir `glob` oder `os`.

In [5]:
import glob as g

# Alle Excel-Dateien im Ordner finden
excel_files = g.glob('*.xlsx')
print(excel_files)

# Daten aus allen Dateien in einen DataFrame zusammenführen
all_data = pd.DataFrame()

for file in excel_files:
    df = pd.read_excel(file, engine='openpyxl')
    all_data = pd.concat([all_data, df], ignore_index=True)

display(all_data)

['report_2023_01.xlsx', 'report_2023_02.xlsx', 'report_2023_03.xlsx', 'verkaufsdaten.xlsx']


,Monat,Produkt,Region,Anzahl,Preis_pro_Einheit,Datum,Kategorie,Preis,Menge
0,January,Consulting,EMEA,"1,0",1000.0,NaN,NaN,NaN,NaN
1,January,Service A,DACH,6.0,100.0,NaN,NaN,NaN,NaN
2,January,Consulting,US,zu teuer,500.0,NaN,NaN,NaN,NaN
3,January,Service A,EMEA,10,500.0,NaN,NaN,NaN,NaN
4,January,Service B,EMEA,8,1000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
57,March,Consulting,APAC,8,100.0,NaN,NaN,NaN,NaN
58,March,Lizenz,DACH,7,200.0,NaN,NaN,NaN,NaN
59,March,Service A,APAC,8,500.0,NaN,NaN,NaN,NaN
60,NaN,Laptop,NaN,NaN,NaN,2024-01-01,Elektronik,1200.0,1.0


## 6. Datenanalyse nach dem Laden

Jetzt können wir mit Pandas die üblichen Analysen durchführen:
- Überblick (`head`, `info`, `describe`)
- Filtern
- Gruppierungen (`groupby`) und Aggregationen
- Neue Spalten erstellen (z.B. Umsatz)

In [6]:
# Beispiel: Umsatz berechnen
all_data['Umsatz'] = all_data['Preis'] * all_data['Menge']

# 1. Schritt: Kommas durch Punkte ersetzen (falls nötig)
# .astype(str) stellt sicher, dass wir wirklich auf Text arbeiten, bevor wir ersetzen
all_data['Anzahl'] = all_data['Anzahl'].astype(str).str.replace(',', '.')

# 2. Schritt: In echte Zahlen umwandeln
# errors='coerce' ist der wichtigste Teil: Er verwandelt alles, was keine Zahl ist 
# (z.B. "ausverkauft", leere Zellen oder Tippfehler), in NaN (Not a Number).
# So stürzt dein Skript nicht ab.
all_data['Anzahl'] = pd.to_numeric(all_data['Anzahl'], errors='coerce')

# Optional: NaN (ungültige Werte) durch 0 ersetzen, damit man rechnen kann
all_data['Anzahl'] = all_data['Anzahl'].fillna(0)

# JETZT kannst du deine Rechnung machen
all_data['Bla'] = all_data['Anzahl'] * 10

display(all_data)

# Gesamtumsatz pro Produkt
umsatz_pro_produkt = all_data.groupby('Produkt')['Anzahl'].sum()

,Monat,Produkt,Region,Anzahl,Preis_pro_Einheit,Datum,Kategorie,Preis,Menge,Umsatz,Bla
0,January,Consulting,EMEA,1.0,1000.0,NaN,NaN,NaN,NaN,NaN,10.0
1,January,Service A,DACH,6.0,100.0,NaN,NaN,NaN,NaN,NaN,60.0
2,January,Consulting,US,0.0,500.0,NaN,NaN,NaN,NaN,NaN,0.0
3,January,Service A,EMEA,10.0,500.0,NaN,NaN,NaN,NaN,NaN,100.0
4,January,Service B,EMEA,8.0,1000.0,NaN,NaN,NaN,NaN,NaN,80.0
...,...,...,...,...,...,...,...,...,...,...,...
57,March,Consulting,APAC,8.0,100.0,NaN,NaN,NaN,NaN,NaN,80.0
58,March,Lizenz,DACH,7.0,200.0,NaN,NaN,NaN,NaN,NaN,70.0
59,March,Service A,APAC,8.0,500.0,NaN,NaN,NaN,NaN,NaN,80.0
60,NaN,Laptop,NaN,0.0,NaN,2024-01-01,Elektronik,1200.0,1.0,1200.0,0.0


## 7. Excel-Dateien aktualisieren

Wir können bestehende Excel-Dateien auch mit openpyxl bearbeiten:

In [7]:
# Workbook laden
wb = load_workbook('verkaufsdaten.xlsx')
ws = wb['Verkaufsdaten']

# Neue Zeile hinzufügen
ws.append(['2024-01-05', 'Monitor', 'Elektronik', 300, 2])

# Speichern
wb.save('verkaufsdaten.xlsx')
print("Neue Zeile hinzugefügt.")

Neue Zeile hinzugefügt.


Und Styling ist natürlich auch möglich:

In [8]:

from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Border, Side

# Neues Workbook
wb = Workbook()
ws = wb.active
ws.title = 'StylingTest'

# Beispiel-Daten einfügen
ws.append(['Name', 'Umsatz', 'Stadt'])
ws.append(['Alice', 1200, 'Berlin'])
ws.append(['Bob', 850, 'München'])
ws.append(['Charlie', 430, 'Hamburg'])


# Header fetten, Hintergrundfarbe
header_font = Font(bold=True, color='e0f007')
header_fill = PatternFill(start_color='4F81BD', end_color="4F81BD", fill_type='solid')
border = Border(left=Side(border_style='thin'), right=Side(border_style='thin'),
top=Side(border_style='thin'), bottom=Side(border_style='thin'))
for cell in ws[1]:
    cell.font = header_font
    cell.fill = header_fill
    cell.border = border


# Rahmen für alle Zellen
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.border = border


# Datei speichern
wb.save('styling_beispiel.xlsx')
print('Excel-Styling-Beispiel gespeichert.')

Excel-Styling-Beispiel gespeichert.


## 8. Zusammenfassung

In diesem Notebook haben wir:
1. Excel-Dateien mit `openpyxl` erstellt und gelesen
2. Excel-Daten in Pandas DataFrames geladen
3. Mehrere Excel-Dateien automatisch zusammengeführt (`glob`)
4. Daten analysiert und neue Spalten berechnet
5. Excel-Dateien aktualisiert